# A/B Testing

In [6]:
import math as mt
import numpy as np
import pandas as pd
from scipy.stats import norm

## 2 Experiment Overview

**Experiment Name:** "Free Trial" Screener. <br>
The following is an experiment conducted by Udacity, a website dedicated to online teaching online
## 2.1 Description of Experimented Change 

* In the experiment, Udacity tested a change where if the student clicked "start free trial", they were **asked how much time they had available to devote to the course**
* If the student indicated 5 or more hours per week, they would be taken through the checkout process as usual. If they indicated fewer than 5 hours per week, a message would appear indicating that Udacity courses usually require a greater time commitment for successful completion, and suggesting that the student might like to access the course materials for free.
* At this point, the student would have the option to continue enrolling in the free trial, or access the course materials for free instead. 

## 2.2 Experiment Hypothesis 
The hypothesis was that this change will **set clearer expectations**, thus reducing the number of frustrated students who left the free trial because they didn't have enough time. If this hypothesis held true, Udacity could improve the overall student experience and improve coaches' capacity to support students.

### 2.3 Experiment Details
The unit of diversion is a cookie instead of a user-id to track both enrolled and unenrolled students

## 3 Metric Choice 

### 3.1 Invariate Metrics - Sanity Checks <a class="anchor" id="invariate"></a>

| Metric Name  | Metric Formula  | $Dmin$  | Notation |
|:-:|:-:|:-:|:-:|
| Number of Cookies in Course Overview Page  | # unique daily cookies on page | 3000 cookies  | $C_k$ |
| Number of Clicks on Free Trial Button  | # unique daily cookies who clicked  | 240 clicks | $C_l$ |
| Free Trial button Click-Through-Probability  | $\frac{C_l}{C_k}$ | 0.01  | $CTP$ | 

### 3.2 Evaluation Metrics - Performance Indicators
| Metric Name  | Metric Formula  | $Dmin$  | Notation |
|:-:|:-:|:-:|:-:|
| Gross Conversion   |  $\frac{enrolled}{C_l}$  | 0.01  | $C_{Gross}$|
| Retention   | $\frac{paid}{enrolled}$  | 0.01  | $R$|
| Net Conversion  |  $\frac{paid}{C_l}$  | 0.0075 | $C_{Net}$|

## 4 Baseline values of metrics 

### 4.1 Obtaining intial values of each metric from Website Traffic

| Item | Description  | Estimator  |
|:-:|:-:|:-:|
| Number of cookies | Daily unique cookies to view course overview page  | 40,000  |
| Number of clicks | Daily unique cookies to click Free Trial button  | 3,200 |
| Number of enrollments | Free Trial enrollments per day  | 660  |
| CTP | CTP on Free Trial button  | 0.08  |
| Gross Conversion | Probability of enrolling, given a click  | 0.20625  |
| Retention | Probability of payment, given enrollment  | 0.53  |
| Net Conversion | Probability of payment, given click  | 0.109313 |

In [7]:
#Let's place this estimators into a dictionary for ease of use later
baseline = {"Cookies":40000,"Clicks":3200,"Enrollments":660,"CTP":0.08,"GConversion":0.20625,
           "Retention":0.53,"NConversion":0.109313}
baseline

{'Cookies': 40000,
 'Clicks': 3200,
 'Enrollments': 660,
 'CTP': 0.08,
 'GConversion': 0.20625,
 'Retention': 0.53,
 'NConversion': 0.109313}

### 4.2 Estimating Standard Deviation of the Evaluation Metrics with sample size 5000 cookies 

#### 4.2.1 Scaling Collected Data

In [8]:
#Scale The counts estimates
baseline["Cookies"] = 5000
baseline["Clicks"]=baseline["Clicks"]*(5000/40000)
baseline["Enrollments"]=baseline["Enrollments"]*(5000/40000)
baseline

{'Cookies': 5000,
 'Clicks': 400.0,
 'Enrollments': 82.5,
 'CTP': 0.08,
 'GConversion': 0.20625,
 'Retention': 0.53,
 'NConversion': 0.109313}

#### 4.2.2 Estimating Analytically
In order to estimate variance analytically, we can assume metrics which are probabilities ($\hat{p}$) are binomially distributed, so we can use this formula for the standard deviation: <br>
<center><font size="4">$SD=\sqrt{\frac{\hat{p}*(1-\hat{p})}{n}}$</font></center><br>
This assumption is only valid when the **unit of diversion** of the experiment is equal to the **unit of analysis** (the denominator of the metric formula). In the cases when this is not valid, the actual variance might be different and it is recommended to estimate it empirically.

For each metric, we need to plug two variables into the formula: <br>
$\hat{p}$ - baseline probability of the event to occur <br>
$ n $ - sample size <br>

* **Gross Conversion** 

In [9]:
# Let's get the p and n we need for Gross Conversion (GC)
# and compute the Stansard Deviation(sd) rounded to 4 decimal digits.
GC={}
GC["d_min"]=0.01
GC["p"]=baseline["GConversion"]
#p is given in this case - or we could calculate it from enrollments/clicks
GC["n"]=baseline["Clicks"]
GC["sd"]=round(mt.sqrt((GC["p"]*(1-GC["p"]))/GC["n"]),4)
GC["sd"]

0.0202

* **Retention**

In [10]:
# Let's get the p and n we need for Retention(R)
# and compute the Stansard Deviation(sd) rounded to 4 decimal digits.
R={}
R["d_min"]=0.01
R["p"]=baseline["Retention"]
R["n"]=baseline["Enrollments"]
R["sd"]=round(mt.sqrt((R["p"]*(1-R["p"]))/R["n"]),4)
R["sd"]

0.0549

* **Net Conversion** 

In [11]:
# Let's get the p and n we need for Net Conversion (NC)
# and compute the Standard Deviation (sd) rounded to 4 decimal digits.
NC={}
NC["d_min"]=0.0075
NC["p"]=baseline["NConversion"]
NC["n"]=baseline["Clicks"]
NC["sd"]=round(mt.sqrt((NC["p"]*(1-NC["p"]))/NC["n"]),4)
NC["sd"]

0.0156

## 5 Experiment Sizing <a class="anchor" id="sizing"></a>

The question we are answering in this section is, what minimum number of samples do we need for our experiment to have enought statistical power $\beta=0.2$ & significance $\alpha=0.05$? This amount will be divided into tthe two groups: control and experiment.

Number of samples = total page views (cookies who viewed the course overview page)

The minimum sample size for control and experiment groups, which provides probability of Type I Error $\alpha$, Power $1-\beta$, detectable effect $d$ and baseline conversion rate $p$ (simple hypothesis $H_0 : P_{cont} - P_{exp} = 0$ against simple alternative $H_A : P_{cont} - P_{exp} = d$  is:

---
<center> <font size="5"> $n = \frac{(Z_{1-\frac{\alpha}{2}}sd_1 + Z_{1-\beta}sd_2)^2}{d^2}$</font>, with: <br><br>
    $sd_1 = \sqrt{p(1-p)+p(1-p)}$<br><br>
    $sd_2 = \sqrt{p(1-p)+(p+d)(1-(1-(p+d))}$ </center><br>

-----

### 5.2 Calculate Sample Size per Metric

In [1]:
def get_sampSize(alpha,beta,d,p):
    n=pow((norm.ppf(1-alpha/2)*mt.sqrt(2*p*(1-p))+norm.ppf(1-beta)*mt.sqrt(p*(1-p)+(p+d)*(1-(p+d)))),2)/pow(d,2)
    return n

* **Gross Conversion**

In [44]:
# Let's get an integer value for simplicity
GC["SampSize"]=round(get_sampSize(0.05,0.2,GC["d"],GC["p"]))
GC["SampSize"]

25835

This means we need at least 25,835 cookies who click the Free Trial button - per group! That means that if we got 400 clicks out of 5000 pageviews (`400/5000 = 0.08`) -> So, we are going to need `GC["SampSize"]/0.08 = 322,938` pageviews, again ; per group! Finally, the total amount of samples per the Gross Conversion metric is:

In [18]:
GC["SampSize"]=round(GC["SampSize"]/0.08*2)
GC["SampSize"]

645875

* **Retention**

In [27]:
# Getting a nice integer value
R["SampSize"]=round(get_sampSize(get_sds(R["p"],R["d"]),0.05,0.2,R["d"]))
R["SampSize"]

39087

This means that we need 39,087 users who enrolled per group! We have to first convert this to cookies who clicked, and then to cookies who viewed the page, then finally to multipky by two for both groups.

In [28]:
R["SampSize"]=round(R["SampSize"]/0.08/0.20625*2)
R["SampSize"]

4737818

This takes us as high as over 4 million page views total, this is practically impossible because we know we get about 40,000 a day, this would take well over 100 days. This means we have to drop this metric and not continue to work with it because results from our experiment (which is much smaller) will be biased.
* **Net Conversion**

In [32]:
# Getting a nice integer value
NC["SampSize"]=round(get_sampSize(get_sds(NC["p"],NC["d"]),0.05,0.2,NC["d"]))
NC["SampSize"]

27413

So, needing 27,413 cookies who click per group takes us all the way up to:

In [33]:
NC["SampSize"]=round(NC["SampSize"]/0.08*2)
NC["SampSize"]

685325

We are all the way up to **685,325** cookies who view the page. This is more than what was needed for Gross Conversion, so this will be our number. Assuming we take 80% of each days pageviews, the data collection period for this experiment (the period in which the experiment is revealed) will be about **3 weeks**


## 6 Analyzing Collected Data

### 6.1 Loading collected data

In [54]:
# we use pandas to load datasets
control=pd.read_csv("./data/control_data.csv")
experiment=pd.read_csv("./data/experiment_data.csv")
control

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7723,687,134.0,70.0
1,"Sun, Oct 12",9102,779,147.0,70.0
2,"Mon, Oct 13",10511,909,167.0,95.0
3,"Tue, Oct 14",9871,836,156.0,105.0
4,"Wed, Oct 15",10014,837,163.0,64.0
5,"Thu, Oct 16",9670,823,138.0,82.0
6,"Fri, Oct 17",9008,748,146.0,76.0
7,"Sat, Oct 18",7434,632,110.0,70.0
8,"Sun, Oct 19",8459,691,131.0,60.0
9,"Mon, Oct 20",10667,861,165.0,97.0


### 6.2 Sanity Checks
These checks help verify that the experiment was conducted as expected and that other factors did not influence the data which we collected. This also makes sure that data collection was correct.

We have 3 Invariant metrics:: 
* Number of Cookies in Course Overview Page
* Number of Clicks on Free Trial Button
* Free Trial button Click-Through-Probability

Two of these metrics are simple counts like number of cookies or number of clicks and the third is a probability (CTP). We will use two different ways of checking whether these obsereved values are like we expect (if in fact the experiment was not damaged.

#### 6.2.1 Sanity Checks for differences between counts
* **Number of cookies who viewed the course overview page**

In [36]:
pageviews_cont=control['Pageviews'].sum()
pageviews_exp=experiment['Pageviews'].sum()
pageviews_total=pageviews_cont+pageviews_exp
print ("number of pageviews in control:", pageviews_cont)
print ("number of Pageviews in experiment:" ,pageviews_exp)

number of pageviews in control: 345543
number of Pageviews in experiment: 344660




What we want to test is whether our observed $\hat{p}$ (number of samples in control divided by total number of damples in both groups) is not significantly different than $p=0.5$. In order to do that we can calculate the margin of error acceptable at a 95% confidence level:
<center> <font size="4"> $ ME=Z_{1-\frac{\alpha}{2}}SD$ </font></center>

Finally, a confidence interval can be derived to tell us in which range an observed $p$ can exist and be acceptable as "the same" as the expected value.

---
<center> <font size="4"> $ CI=[\hat{p}-ME,\hat{p}+ME]$ </font></center>

When our obsereved $\hat{p}$ is within this range, all is well and the test was passed.

In [45]:
p=0.5
alpha=0.05
p_hat=round(pageviews_cont/(pageviews_total),4)
sd=mt.sqrt(p*(1-p)/(pageviews_total))
ME=round(get_z_score(1-(alpha/2))*sd,4)
print ("The confidence interval is between",p-ME,"and",p+ME,"; Is",p_hat,"inside this range?")

The confidence interval is between 0.4988 and 0.5012 ; Is 0.5006 inside this range?


Our observed $\hat{p}$ is inside this range which means the difference in number of samples between groups is expected.
* **Number of cookies who clicked the Free Trial Button**

In [39]:
clicks_cont=control['Clicks'].sum()
clicks_exp=experiment['Clicks'].sum()
clicks_total=clicks_cont+clicks_exp

p_hat=round(clicks_cont/clicks_total,4)
sd=mt.sqrt(p*(1-p)/clicks_total)
ME=round(get_z_score(1-(alpha/2))*sd,4)
print ("The confidence interval is between",p-ME,"and",p+ME,"; Is",p_hat,"inside this range?")

The confidence interval is between 0.4959 and 0.5041 ; Is 0.5005 inside this range?


Our observed $\hat{p}$ is inside this range which means the difference in number of clicks between groups is expected.

#### 6.2.2 Sanity Checks for differences between probabilities <a class="anchor" id="check_prob"></a>
* **Click-through-probability of the Free Trial Button**
In this case, we want to make sure the proportion of clicks given a pageview (our observed CTP) is about the same in both groups (since this was not expected to change due to the experiment). In order to check this out we will calculate the CTP in each group and calculate a confidence interval for the expected difference between them. 

In other words, we expect to see no difference ($CTP_{exp}-CTP_{cont}=0$), with an acceptable margin of error, dictated by our calculated confidence interval. The changes we should notice are for the calculation of the standard error - which in this case is a pooled standard error.

<center><font size="4">$SD_{pool}=\sqrt{\hat{p_{pool}}(1-\hat{p_{pool}}(\frac{1}{N_{cont}}+\frac{1}{N_{exp}})}$</font></center>
with <br> <center><font size="5"> $\hat{p_{pool}}=\frac{x_{cont}+x_{exp}}{N_{cont}+N_{exp}}$ </font></center>
<p></p>
<center> <font size="4"> $ ME=Z_{1-\frac{\alpha}{2}}SD$ </font></center>
<p></p>
<center> <font size="4"> $ CI=[\hat{p}-ME,\hat{p}+ME]$ </font></center>

We should understand that CTP is a proportion in a population (amount of events x in a population n) like the amount of clicks out of the amount of pageviews..

In [41]:
ctp_cont=clicks_cont/pageviews_cont
ctp_exp=clicks_exp/pageviews_exp
d_hat=round(ctp_exp-ctp_cont,4)
p_pooled=clicks_total/pageviews_total
sd_pooled=mt.sqrt(p_pooled*(1-p_pooled)*(1/pageviews_cont+1/pageviews_exp))
ME=round(get_z_score(1-(alpha/2))*sd_pooled,4)
print ("The confidence interval is between",0-ME,"and",0+ME,"; Is",d_hat,"within this range?")

The confidence interval is between -0.0013 and 0.0013 ; Is 0.0001 within this range?


Test Passed!!

### 6.3 Examining effect size
The next step is looking at the changes between the control and experiment groups with regard to our evaluation metrics to make sure the difference is there, that it is statistically significant and most importantly practically significant.

* **Gross Conversion**

In [26]:
#Calculate Gross Conversion for each control & expreimental study

#Number of enrollments
enrollments_cont=control["Enrollments"].sum()
enrollments_exp=experiment["Enrollments"].sum()

#Number of clicks
clicks_cont=control["Clicks"].loc[control["Enrollments"].notnull()].sum()
clicks_exp=experiment["Clicks"].loc[experiment["Enrollments"].notnull()].sum()

#Gross Convertion for each control & experimental group (Number of enrollments/Number of Clicks)
GC_cont=enrollments_cont/clicks_cont
GC_exp=enrollments_exp/clicks_exp

#Gross Conversion doffereance between experiment & control
GC_diff=round(GC_exp-GC_cont,4)

#Margin of error Calculation
    #Gross conversion for entire Dataset
GC_pooled=(enrollments_cont+enrollments_exp)/(clicks_cont+clicks_exp)

    #Gross Conversion Sd for entire Dataset
GC_sd_pooled=mt.sqrt(GC_pooled*(1-GC_pooled)*(1/clicks_cont+1/clicks_exp))

    #Margin of error
GC_ME=round(get_z_score(1-alpha/2)*GC_sd_pooled,4)

print("The change due to the experiment is",GC_diff*100,"%")
print("Confidence Interval: [",GC_diff-GC_ME,",",GC_diff+GC_ME,"]")
print ("The change is statistically significant if the CI doesn't include 0. In that case, it is practically significant if",-GC["d_min"],"is not in the CI as well.")

The change due to the experiment is -2.06 %
Confidence Interval: [ -0.0292 , -0.012 ]
The change is statistically significant if the CI doesn't include 0. In that case, it is practically significant if -0.01 is not in the CI as well.


According to this result there was a change due to the experiment, that change was both statistically and practically significant. 
We have a negative change of 2.06%. This means the Gross Conversion rate of the experiment group (the one exposed to the change, i.e. asked how many hours they can devote to studying) has decreased as expected by 2% and this change was significant.
* **Net Conversion** 

In [46]:
#Number of payments 
payments_cont=control["Payments"].sum()
payments_exp=experiment["Payments"].sum()

#Net Conversion (Number or payment/number of clicks)
NC_cont=payments_cont/clicks_cont
NC_exp=payments_exp/clicks_exp

#Net conversion of the entire Dataset
NC_pooled=(payments_cont+payments_exp)/(clicks_cont+clicks_exp)

#Net conversion Sd of the entire Dataset
NC_sd_pooled=mt.sqrt(NC_pooled*(1-NC_pooled)*(1/clicks_cont+1/clicks_exp))

#Margin of Error of entire Dataset
NC_ME=round(get_z_score(1-alpha/2)*NC_sd_pooled,4)

#Net conversion Difference between experinental & control
NC_diff=round(NC_exp-NC_cont,4)
print("The change due to the experiment is",NC_diff*100,"%")
print("Confidence Interval: [",NC_diff-NC_ME,",",NC_diff+NC_ME,"]")
print ("The change is statistically significant if the CI doesn't include 0. In that case, it is practically significant if",NC["d_min"],"is not in the CI as well.")

The change due to the experiment is -0.3 %
Confidence Interval: [ -0.0072 , 0.0011999999999999997 ]
The change is statistically significant if the CI doesn't include 0. In that case, it is practically significant if 0.0075 is not in the CI as well.


In this case we got a change size of less than a 0.5%, a very small decrease which is not statistically significant, and as such not practically significant.

## 6.4 Double check with Sign Tests 
Compute the metric's value per day and then count on how many days the metric was lower in the experiment group and this will be the number of succssesses for our binomial variable. Once this is defined we can look at the proportion of days of success out of all the available days.

### 6.4.1 Data Preparation 

In [49]:
#let's first create the dataset we need for this:
# start by merging the two datasets
full=control.join(other=experiment,how="inner",lsuffix="_cont",rsuffix="_exp")
#Let's look at what we got
full.count()

Date_cont           37
Pageviews_cont      37
Clicks_cont         37
Enrollments_cont    23
Payments_cont       23
Date_exp            37
Pageviews_exp       37
Clicks_exp          37
Enrollments_exp     23
Payments_exp        23
dtype: int64

In [50]:
#now we only need the complete data records
full=full.loc[full["Enrollments_cont"].notnull()]
full.count()

Date_cont           23
Pageviews_cont      23
Clicks_cont         23
Enrollments_cont    23
Payments_cont       23
Date_exp            23
Pageviews_exp       23
Clicks_exp          23
Enrollments_exp     23
Payments_exp        23
dtype: int64

In [55]:
# Perfect! Now, derive a new column for each metric, so we have it's daily values
# We need a 1 if the experiment value is greater than the control value=
x=full['Enrollments_cont']/full['Clicks_cont']
y=full['Enrollments_exp']/full['Clicks_exp']
full['GC'] = np.where(x<y,1,0)
# The same now for net conversion
z=full['Payments_cont']/full['Clicks_cont']
w=full['Payments_exp']/full['Clicks_exp']
full['NC'] = np.where(z<w,1,0)
full

,Date_cont,Pageviews_cont,Clicks_cont,Enrollments_cont,Payments_cont,Date_exp,Pageviews_exp,Clicks_exp,Enrollments_exp,Payments_exp,GC,NC
0,"Sat, Oct 11",7723,687,134.0,70.0,"Sat, Oct 11",7716,686,105.0,34.0,0,0
1,"Sun, Oct 12",9102,779,147.0,70.0,"Sun, Oct 12",9288,785,116.0,91.0,0,1
2,"Mon, Oct 13",10511,909,167.0,95.0,"Mon, Oct 13",10480,884,145.0,79.0,0,0
3,"Tue, Oct 14",9871,836,156.0,105.0,"Tue, Oct 14",9867,827,138.0,92.0,0,0
4,"Wed, Oct 15",10014,837,163.0,64.0,"Wed, Oct 15",9793,832,140.0,94.0,0,1
5,"Thu, Oct 16",9670,823,138.0,82.0,"Thu, Oct 16",9500,788,129.0,61.0,0,0
6,"Fri, Oct 17",9008,748,146.0,76.0,"Fri, Oct 17",9088,780,127.0,44.0,0,0
7,"Sat, Oct 18",7434,632,110.0,70.0,"Sat, Oct 18",7664,652,94.0,62.0,0,0
8,"Sun, Oct 19",8459,691,131.0,60.0,"Sun, Oct 19",8434,697,120.0,77.0,0,1
9,"Mon, Oct 20",10667,861,165.0,97.0,"Mon, Oct 20",10496,860,153.0,98.0,0,1


In [56]:
GC_x=full.GC[full["GC"]==1].count()
NC_x=full.NC[full["NC"]==1].count()
n=full.NC.count()
print("No. of cases for GC:",GC_x,'\n',
      "No. of cases for NC:",NC_x,'\n',
      "No. of total cases",n)

No. of cases for GC: 4 
 No. of cases for NC: 10 
 No. of total cases 23


### 6.4.2 Building a Sign Test



In [58]:
#first a function for calculating probability of x=number of successes
def get_prob(x,n):
    p=round(mt.factorial(n)/(mt.factorial(x)*mt.factorial(n-x))*0.5**x*0.5**(n-x),4)
    return p
#next a function to compute the pvalue from probabilities of maximum x
def get_2side_pvalue(x,n):
    p=0
    for i in range(0,x+1):
        p=p+get_prob(i,n)
    return 2*p

In [59]:
print ("GC Change is significant if",get_2side_pvalue(GC_x,n),"is smaller than 0.05")
print ("NC Change is significant if",get_2side_pvalue(NC_x,n),"is smaller than 0.05")

GC Change is significant if 0.0026000000000000003 is smaller than 0.05
NC Change is significant if 0.6774 is smaller than 0.05


We get the same conclusions as we got from our effect size calculation: the change in Gross conversion was indeed significant, while the change in Net conversion was not.

## 7 Conclusions & Recommendations
We can see from the results that the actual underlying goal we had was not reached (increase net conversion), we can only recommend to not continue with change. 
